In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate


KeyboardInterrupt: ignored

Google Drive.

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


Path of original image and style reference image.

In [0]:
import os

base_image_path = 'drive/style_transfer/vedioFrame/665.jpg'
style_reference_image_path = 'drive/style_transfer/nahan.jpg'
result_prefix = 'drive/style_transfer/frame665/'

settings: weights of the different loss components

In [0]:
iterations = 2000

content_weight = 1
style_weight = 1000

imensions of the generated picture

In [0]:
from keras.preprocessing.image import load_img, save_img, img_to_array

width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

Using TensorFlow backend.


function to open, resize and format pictures into appropriate tensors

In [0]:
from keras.applications import vgg19
import numpy as np

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols)) # load the image and resize it to (img_nrows, img_ncols)
    img = img_to_array(img) #  to shape(img_nrows, img_ncols, 3)
    img = np.expand_dims(img, axis=0) # the expanded dimention is used for sample_number for a batch of samples (samples, width,height,channels)
    img = vgg19.preprocess_input(img) # adequate the image to the format the model requires (normalized the pixels between -1 to +1,)
    return img

function to convert a tensor into a valid image

In [0]:
from keras import backend as K

def deprocess_image(x):
    if K.image_data_format() == 'channels_first': # For “theano”, it should be “channels_first”. For "tensorflow "or "cntk" backends, it should be “channels_last”.
        x = x.reshape((3, img_nrows, img_ncols))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

get tensor representations of our images

In [0]:
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

Instructions for updating:
Colocations handled automatically by placer.


this will contain our generated image

In [0]:
if K.image_data_format() == 'channels_first':
    combination_image = K.placeholder((1, 3, img_nrows, img_ncols)) 
else:
    combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

combine the 3 images into a single Keras tensor

In [0]:
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

build the VGG19 network with our 3 images as input the model will be loaded with pre-trained ImageNet weights

In [0]:
model = vgg19.VGG19(input_tensor=input_tensor,weights='imagenet', include_top=False)
print('Model loaded.')

Model loaded.


get the symbolic outputs of each "key" layer (we gave them unique names).

In [0]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

compute the neural style loss first we need to define 4 util functions

the gram matrix of an image tensor (feature-wise outer product)

In [0]:
def gram_matrix(x):
    assert K.ndim(x) == 3
    if K.image_data_format() == 'channels_first':
        features = K.batch_flatten(x)
    else:
        features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

The "style loss" is designed to maintain the style of the reference image in the generated image.
It is based on the gram matrices (which capture style) of feature maps from the style reference image and from the generated image.

In [0]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

An auxiliary loss function designed to maintain the "content" of the base image in the generated image

In [0]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

The 3rd loss function, total variation loss, designed to keep the generated image locally coherent

In [0]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    if K.image_data_format() == 'channels_first':
        a = K.square(
            x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, 1:, :img_ncols - 1])
        b = K.square(
            x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, :img_nrows - 1, 1:])
    else:
        a = K.square(
            x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
        b = K.square(
            x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

Combine these loss functions into a single scalar.

In [0]:
loss = K.variable(0.0)
layer_features = outputs_dict['block4_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(base_image_features,combination_features)

style loss

In [0]:
feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(feature_layers)) * sl

get the gradients of the generated image wrt the loss

In [0]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

In [0]:
def eval_loss_and_grads(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((1, 3, img_nrows, img_ncols))
    else:
        x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

This Evaluator class makes it possible to compute loss and gradients in one pass while retrieving them via two separate functions, "loss" and "grads". This is done because scipy.optimize requires separate functions for loss and gradients, but computing them separately would be inefficient.

In [0]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

In [0]:
evaluator = Evaluator()

run scipy-based optimization (L-BFGS) over the pixels of the generated image so as to minimize the neural style loss

In [0]:
from scipy.optimize import fmin_l_bfgs_b
import time

#x = vgg19.preprocess_input(np.random.uniform(0, 255,size=(1,img_nrows,img_ncols, 3)))
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix + '_at_iteration_%d.jpg'%i
    save_img(fname, img)
    print('Image saved as', fname)
    end_time = time.time()
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 3252866000000.0
Image saved as drive/style_transfer/frame665/_at_iteration_0.jpg
Iteration 0 completed in 11s
Start of iteration 1
Current loss value: 1586672300000.0
Image saved as drive/style_transfer/frame665/_at_iteration_1.jpg
Iteration 1 completed in 11s
Start of iteration 2
Current loss value: 1298857000000.0
Image saved as drive/style_transfer/frame665/_at_iteration_2.jpg
Iteration 2 completed in 12s
Start of iteration 3
Current loss value: 1170937700000.0
Image saved as drive/style_transfer/frame665/_at_iteration_3.jpg
Iteration 3 completed in 10s
Start of iteration 4
Current loss value: 1083267200000.0
Image saved as drive/style_transfer/frame665/_at_iteration_4.jpg
Iteration 4 completed in 11s
Start of iteration 5
Current loss value: 1017841000000.0
Image saved as drive/style_transfer/frame665/_at_iteration_5.jpg
Iteration 5 completed in 11s
Start of iteration 6
Current loss value: 969462050000.0
Image saved as drive/style_transfer/fr

KeyboardInterrupt: ignored